In [2]:
import os
import io
import pandas as pd

"""
What does this do?
Take flattened Bionano segment annotation table and cut out segment sequence from
reference to align against phased assemblies (prerequisite to compute concordance
for 3q29 between Bionano and phased assemblies).
"""


chr3_fasta = '/home/local/work/data/hgsvc/roi/dotplots_chr3/GRCh38_HGSVC2_noalt.chr3.fasta'

bng_table = pd.read_csv(
    '/home/local/work/data/hgsvc/roi/20201109_3q29_bng_segments.flat.tsv',
    sep='\t',
    header=0
)

refseq = ''
with open(chr3_fasta, 'r') as fasta:
    for line in fasta:
        if line.startswith('>'):
            continue
        refseq += line.strip()
        
bng_table = bng_table.loc[bng_table['contig_id'] == 'reference', :].copy()

all_colors = io.StringIO()

line_length = 80
out_path = '/home/local/work/data/hgsvc/roi/dotplots_chr3/chr3_segments'
for position, (idx, row) in enumerate(bng_table.iterrows(), start=1):
    start = row['segment_start']
    end = row['segment_end']
    name = row['segment_color']
    segment_seq = refseq[start:end]
    
    header = '>chr3q29_{}_{}'.format(name, position)
    out_file = os.path.join(out_path, 'hg38_chr3q29_{}_{}.fasta'.format(name, position))
    dumped = 0
    with open(out_file, 'w') as dump:
        _ = dump.write(header + '\n')
        _ = all_colors.write(header+'\n')
        for pos in range(0, len(segment_seq)//line_length + 1):
            written = dump.write(segment_seq[pos*line_length:pos*line_length+line_length])
            _ = all_colors.write(segment_seq[pos*line_length:pos*line_length+line_length] + '\n')
            dumped += written
            _ = dump.write('\n')
    all_colors.write('\n')
    print(name, position, ' - ', dumped)
    
with open(os.path.join(out_path, 'hg38_chr3q29_all_segments.fasta'), 'w') as dump:
    _ = dump.write(all_colors.getvalue())
        


magenta 1  -  26760
blue 2  -  7479
yellow 3  -  5780
red 4  -  40729
orange 5  -  15890
darkblue 6  -  139585
purple 7  -  35781
yellow 8  -  5755
blue 9  -  6382
